In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
import sys
import pickle
from datetime import datetime
%matplotlib inline
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.utils import class_weight
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import numpy as np
import warnings
warnings.filterwarnings("ignore")

**Let's check the test set composition**

In [ ]:
test_set = pd.read_csv('../input/commonlitreadabilityprize/test.csv')


In [ ]:
test_set.head()

In [ ]:
test_set.shape

**Let's start working on the train set**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
import sys
import pickle
from datetime import datetime
%matplotlib inline
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.utils import class_weight
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('../input/commonlitreadabilityprize/train.csv')

In [ ]:
df.info()

**Let's focus on "excerpt" and "target"**

In [ ]:
df.head()

In [ ]:
df[["excerpt", "target"]].info()

In [ ]:
df[["excerpt", "target"]].head()

In [ ]:
df['target'].hist(bins=40)
plt.title('Distribuzione variabile target')
plt.ylabel('Conteggio')
plt.show()

**Let's focus on null values**

In [ ]:
df.isna().sum()

In [ ]:
df.isna().sum().sum()

**Correlation**

In [ ]:
df.corr()

**Let's save standard error for later and drop url_legal and license columns**

In [ ]:
stde = df['standard_error']

In [ ]:
df = df.drop(columns = ['url_legal','license', 'id', 'standard_error'])

**Let's transform continue target in categorical one**

In [ ]:
print(df['target'].min(),df['target'].max())
numberOfStars=6
mybin=((df['target'].max())-(df['target'].min()-1))/numberOfStars
bins=[]
for i in range(numberOfStars+1):
    bins.append((df['target'].min()-1)+i*mybin)
print (bins)

stars = pd.cut(df.target,bins=6,labels=[0,1,2,3,4,5])#stelle mappate fra 0 e 4 perchè il to_categorical() di keras parte da indice 0
df.insert(2,'stars',stars)
df["stars"] = pd.to_numeric(df["stars"])


In [ ]:
df["stars"]

In [ ]:
df.stars.shape

In [ ]:
df.head()

In [ ]:
df.shape

**Dropping target**


In [ ]:
df = df.drop(['target'], axis=1)

In [ ]:
df.head()

In [ ]:
df.stars.value_counts()

**Unbalanced class! We are going to perform smote**

**Features engineering**

In [ ]:
X= df['excerpt']
y = df['stars']

In [ ]:
#splitto dataset in training e test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42,stratify=y)#stratifico rispetto y


print(type(X_train))
print (X_train[0])

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2# per features selection
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

In [ ]:
vect = CountVectorizer(min_df=1,ngram_range=(1,2))  # tokenization and feature extraction (fa tokenizzazione di default conterrà contatori )
                                  #min_df=5 elimino parole sulla coda lunga (numerose e poco frequenti count<5)
                                  #ngram_range=(1,2) considero solo le singole parole e  bigrammi, non trigrammi 
vect.fit(X_train)
X_train = vect.transform(X_train)
X_test =vect.transform(X_test)

In [ ]:
sel = SelectKBest(chi2, k=5000)  # feature selection  faccio features selection in maniera supervisionata gli passo y_train
sel.fit(X_train,y_train)
X_train = sel.transform(X_train)
X_test = sel.transform(X_test)

In [ ]:
tfidf = TfidfTransformer()  # weighting
tfidf.fit(X_train)
X_train = tfidf.transform(X_train)
X_test =tfidf.transform(X_test)


In [ ]:
print(X_train)

In [ ]:
print(X_test)

In [ ]:
# la classe 4 è sbilanciata, decido quindi di bilanciarla computando la class weight che poi usero come parametro nel fitting della rete neurale
my_class_weights = class_weight.compute_class_weight('balanced', # per avere dataset perfettamente bilanciato (per il training) senza però toccare niente sul dataset
                                                  np.unique(y_train),
                                                  y_train)

for _class in [0 ,1,2,3,4]:
    print(f"Weight of instances in class {_class:d}: {my_class_weights[_class]:.2f}")
type(my_class_weights)

**Smote**

In [ ]:
print('Before OverSampling, the shape of train_X: {}'.format(X_train.shape))
print('Before OverSampling, the shape of train_y: {} \n'.format(y_train.shape))



print("Before OverSampling, counts of label '5': {}".format(sum(y_train == 5)))
print("Before OverSampling, counts of label '4': {}".format(sum(y_train == 4)))
print("Before OverSampling, counts of label '3': {}".format(sum(y_train == 3)))
print("Before OverSampling, counts of label '2': {}".format(sum(y_train == 2)))
print("Before OverSampling, counts of label '1': {}".format(sum(y_train == 1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train == 0)))
  
# import SMOTE module from imblearn library
# pip install imblearn (if you don't have imblearn in your system)
from imblearn.over_sampling import SMOTE

oversample = SMOTE(random_state = 2)
X_train_res, y_train_res = oversample.fit_resample(X_train, y_train)

print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))
  

print("After OverSampling, counts of label '5': {}".format(sum(y_train_res == 5)))
print("After OverSampling, counts of label '4': {}".format(sum(y_train_res == 4)))    
print("After OverSampling, counts of label '3': {}".format(sum(y_train_res == 3)))
print("After OverSampling, counts of label '2': {}".format(sum(y_train_res == 2)))
print("After OverSampling, counts of label '1': {}".format(sum(y_train_res == 1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res == 0)))

## **Decision Tree**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()


clf = DecisionTreeClassifier()

"""param_list = {
    'max_depth': [None, 2, 4, 6, 8, 16, 32],
    'min_samples_leaf': [1, 5, 10, 20, 50, 100],
    'min_samples_split': [2, 5, 10, 20, 50, 100],
    'criterion': ['gini', 'entropy']
}


grid_search = GridSearchCV(clf, param_grid=param_list, cv=5, scoring='accuracy') # ricerca esaustiva fra i parametri
grid_search.fit(X_train_res, y_train_res)# (NB: gli passo X_train,y_train ossia solo training perchè poi utilizzo il modello ritornato)
clf_best_grid = grid_search.best_estimator_ # il miglior calssificatore trovato rispetto all'accuratezza"""

In [ ]:
"""grid_search.best_params_"""

In [ ]:
"""clf_best_grid"""

**Training the model on train_res with best params**

In [ ]:
clf = DecisionTreeClassifier(criterion= 'gini', max_depth= 32, 
                             min_samples_leaf= 1, min_samples_split= 5) 

clf.fit(X_train_res, y_train_res)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
from scikitplot.metrics import plot_roc

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print('Accuracy %s' % accuracy_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
matrix = plot_confusion_matrix(clf, X_test, y_test,
                            #display_labels=le.classes_,
                            cmap=plt.cm.Blues,
                            xticks_rotation=70)
matrix.ax_.set_title('Confusion Matrix')

plt.show()

In [ ]:
plot_roc(y_test, clf.predict_proba(X_test))
plt.show()

## **Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets, metrics, model_selection, svm
from sklearn.metrics import confusion_matrix, accuracy_score 
from sklearn.metrics import f1_score, precision_score, recall_score, fbeta_score
from sklearn.metrics import classification_report, precision_recall_curve
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV # ricerca esaustiva fra i parametri
from sklearn.model_selection import RandomizedSearchCV # ricerca randomica fra i parametri con numero prefissato di candidati

clf = RandomForestClassifier()

"""param_list = {
    'max_depth': [None, 2, 4, 6, 8, 16, 32],
    'min_samples_leaf': [1, 5, 10, 20, 50, 100],
    'min_samples_split': [2, 5, 10, 20, 50, 100],
    'criterion': ['gini', 'entropy'],
    'n_estimators': [10,20,50,100,200]
}


grid_search = GridSearchCV(clf, param_grid=param_list, cv=5, scoring='accuracy') # ricerca esaustiva fra i parametri
grid_search.fit(X_train_res, y_train_res)# (NB: gli passo X_train,y_train ossia solo training perchè poi utilizzo il modello ritornato)
clf_best_grid = grid_search.best_estimator_ # il miglior calssificatore trovato rispetto all'accuratezza"""

In [ ]:
"""grid_search.best_params_"""

In [ ]:
"""clf_best_grid"""

In [ ]:
clf = RandomForestClassifier(criterion='gini',
                       max_depth=None, min_samples_leaf=1,
                       min_samples_split=10,
                       n_estimators=200) 

clf.fit(X_train_res, y_train_res)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
from scikitplot.metrics import plot_roc

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print('Accuracy %s' % accuracy_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
from sklearn.metrics import plot_confusion_matrix

matrix = plot_confusion_matrix(clf, X_test, y_test,
                            #display_labels=le.classes_,
                            cmap=plt.cm.Blues,
                            xticks_rotation=70)
matrix.ax_.set_title('Confusion Matrix')

plt.show()

In [ ]:
plot_roc(y_test, clf.predict_proba(X_test))
plt.show()

## **Gradient Boosting Classifier**

In [ ]:
!pip install xgboost

In [ ]:
from numpy import loadtxt
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.model_selection import GridSearchCV # ricerca esaustiva fra i parametri
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import *
clf = xgb.XGBClassifier()


clf.fit(X_train_res,y_train_res)

In [ ]:
"""clf_best_grid"""

In [ ]:
"""grid_search.best_params_"""

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print('Accuracy %s' % accuracy_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
from sklearn.metrics import plot_confusion_matrix

matrix = plot_confusion_matrix(clf, X_test, y_test,
                            #display_labels=le.classes_,
                            cmap=plt.cm.Blues,
                            xticks_rotation=70)
matrix.ax_.set_title('Confusion Matrix')

plt.show()

In [ ]:
plot_roc(y_test, clf.predict_proba(X_test))
plt.show()

## **Stacking Ensemble Classifier**

In [ ]:
!pip install mlxtend

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import itertools
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn import model_selection
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import metrics

# Classifiers
from sklearn.svm import NuSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingCVClassifier # <- Here is our boy


# Used to ignore warnings generated from StackingCVClassifier
import warnings
warnings.simplefilter('ignore')

In [ ]:
# compare ensemble to each baseline classifier
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from matplotlib import pyplot

In [ ]:
# get a stacking ensemble of models
def get_stacking():
    # define the base models
    level0 = list()
    #level0.append(('lr', LogisticRegression()))
    #level0.append(('knn', KNeighborsClassifier()))
    level0.append(('rf', RandomForestClassifier()))
    #level0.append(('xgb',  xgb.XGBClassifier()))
    #level0.append(('gbc',  GradientBoostingClassifier()))
    #level0.append(('cart', DecisionTreeClassifier()))
    #level0.append(('linearsvm', LinearSVC()))
    level0.append(('svm', SVC()))
    #level0.append(('nusvm', NuSVC()))
    level0.append(('mlp', MLPClassifier()))
    #level0.append(('bayes', GaussianNB()))
    # define meta learner model
    level1 = LogisticRegression()
    # define the stacking ensemble
    model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
    return model
 
# get a list of models to evaluate
def get_models():
    models = dict()
    #models['lr'] = LogisticRegression()
    #models['knn'] = KNeighborsClassifier()
    models['rf'] = RandomForestClassifier()
    #models['xgb'] = xgb.XGBClassifier()
    #models['gbc'] = GradientBoostingClassifier()
    #models['cart'] = DecisionTreeClassifier()
    #models['linearsvm'] = LinearSVC()
    models['svm'] = SVC()
    #models['nusvm'] = NuSVC()
    models['mlp'] = MLPClassifier()
    #models['bayes'] = GaussianNB()
    models['stacking'] = get_stacking()
    return models
 
# evaluate a give model using cross-validation
def evaluate_model(model, X_train_res, y_train_res):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X_train_res, y_train_res, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
    return scores


# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, X_train_res, y_train_res)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

    # plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()


**After running the previous cell to calculate the single models' performances, I received the follwing results.**

* lr 0.708 (0.020)
* knn 0.664 (0.018)
* rf 0.773 (0.018)
* xgb 0.727 (0.018)
* gbc 0.713 (0.017)
* cart 0.615 (0.021)
* linearsvm 0.742 (0.020)
* svm 0.783 (0.016)
* mlp 0.779 (0.015)


**I decided then to keep the best three models and to run again the cell for getting back the single ones' and the stacking one performances**

In [ ]:
# make a prediction with a stacking ensemble
from sklearn.datasets import make_classification
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

# define the base models
level0 = list()
level0.append(('rf', RandomForestClassifier()))
level0.append(('svm', SVC()))
level0.append(('mlp', MLPClassifier()))
# define meta learner model
level1 = LogisticRegression()

# define the stacking ensemble
model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)


# fit the model on training set
model.fit(X_train_res, y_train_res)


In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print('Accuracy %s' % accuracy_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
from sklearn.metrics import plot_confusion_matrix

matrix = plot_confusion_matrix(model, X_test, y_test,
                            #display_labels=le.classes_,
                            cmap=plt.cm.Blues,
                            xticks_rotation=70)
matrix.ax_.set_title('Confusion Matrix')

plt.show()

In [ ]:
plot_roc(y_test, model.predict_proba(X_test))
plt.show()

## I focus in test set. Need first to convert excerpt with tf/idf and the to get predictions ##

In [ ]:
test_set.head()

In [ ]:
dati = test_set['excerpt']

In [ ]:
dati

In [ ]:
X= dati

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2# per features selection
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

In [ ]:
vect = CountVectorizer(min_df=1,ngram_range=(1,2))  # tokenization and feature extraction (fa tokenizzazione di default conterrà contatori )
                                  #min_df=5 elimino parole sulla coda lunga (numerose e poco frequenti count<5)
                                  #ngram_range=(1,2) considero solo le singole parole e  bigrammi, non trigrammi 
vect.fit(X)
X = vect.transform(X)


In [ ]:
tfidf = TfidfTransformer()  # weighting
tfidf.fit(X)
X= tfidf.transform(X)


In [ ]:
X

In [ ]:
print(X)

In [ ]:
y_pred = model.predict(X)

In [ ]:
print(y_pred)